In [2]:
import os
from tqdm import tqdm

import pandas as pd
from parse_tool import PDFProcessor
from text_processing import *


In [15]:
# NAMESPACE = 'SANTAFE_MX5_2023'
NAMESPACE = 'SONATA_DN8_2024'
# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/2024 아이오닉 5 (IONIQ 5) NE.pdf'

# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/디 올 뉴 싼타페 MX5.pdf'
pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/2024 쏘나타 디 엣지 DN8.pdf'
context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'

os.makedirs(context_file_root_path + f'/image/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/csv/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/text/{NAMESPACE}', exist_ok=True)
context_text_dir = context_file_root_path + f'/text/{NAMESPACE}'

fitz_page_start = 6
fitz_page_end = 498

In [16]:
pdf_processor = PDFProcessor(NAMESPACE, pdf_file, fitz_page_start, fitz_page_end, y_range=(39, 575), context_file_root_path = context_file_root_path)

In [17]:
span_list = []
contents_list = []
img_urls = []
csv_urls = []
contents_type = []
block_nums = []
while 1:
    try:
        if pdf_processor.page_num % 50 == 0:
            print(f"Processing page {pdf_processor.page_num}")
        next(pdf_processor)
        page_result = pdf_processor.parse_pdf()
        
        span_list.extend(page_result['span'])
        contents_list.extend(page_result['contents'])
        csv_urls.extend(page_result['csv_urls'])
        img_urls.extend(page_result['img_urls'])
        contents_type.extend(page_result['contents_type'])
        block_nums.extend(page_result['block_num'])
    except StopIteration as e:
        print('End of processing', e)
        break
    except Exception as e:
        print(f"Error raised: {e}, page_num: {pdf_processor.page_num}")
        break

Processing page 50
Processing page 100
Processing page 150
Processing page 200
Processing page 250
Processing page 300
Processing page 350
Processing page 400
Processing page 450
End of processing 


In [18]:
df = pd.DataFrame({
    'span': span_list,
    'contents': contents_list,
    'csv_urls': csv_urls,
    'img_urls': img_urls,
    'ctype': contents_type,
    'block_num': block_nums
})

df = df[(df['ctype']=='image') | (~df['contents'].isna())]

In [19]:
df.to_csv(f'{context_text_dir}/{NAMESPACE}_result.csv', index=False)
df.to_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet', index=False)

In [20]:
df = pd.read_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet')

In [21]:
df = remove_invalid_row_by_string(df)
df = remove_invalid_row_by_span(df)
df = add_header(df)

In [22]:
block_df = group_by_block(df)
doc_df = group_by_doc(block_df)

In [23]:
doc_df.to_parquet(f'{context_text_dir}/doc_{NAMESPACE}_result.parquet')